In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-11-29 15:58:21,079 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-29 15:58:21,080 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
cache_dir = Path.home() / '/data'
working_dir = Path.home() / '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-11-29 15:58:21,870 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-29 15:58:21,871 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# Configuration

In [ ]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Test New Workflow_LAC",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-7K8W'],
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [ ]:
print(configuration_record)

# Create DatasetBag

In [ ]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

# Feature Manipulation

### Find Features associated with a table

In [ ]:
EA.find_features('Image')

### Instanciate a Feature

In [ ]:
ImageAnnotationFeature =  EA.feature_record_class("Image", "Annotation")
DiagnosisFeature = EA.feature_record_class("Image", "Image_Diagnosis")

### Create a new tag before feature ingestion (if needed)

In [ ]:
EA.add_term("Diagnosis_Tag", "test_feature", description="test feature ingestion routine")

### Crop Image by bounding box


In [6]:
    bb_csv_path, bb_asset_paths = configuration_record.feature_paths('Image', 'Annotation')
    bounding_box_files = [bb_asset_paths['BoundingBox'] / f"box{i}.txt" for i in range(10)]
    for i in range(10):
        bounding_box_files.append(fn := bb_asset_paths['BoundingBox'] / f"box{i}.txt")
        with builtins.open(fn, "w") as fp:
            fp.write(f"Hi there {i}")

    image_bounding_box_feature_list = [ImageBoundingboxFeature(Image=image_rid,
                                                               Execution=fs_execution.execution_rid,
                                                               BoundingBox=asset_rid)
                                       for image_rid, asset_rid in zip(image_rids, itertools.cycle(bounding_box_files))]

NameError: name 'configuration_record' is not defined

# Execution of ML

In [ ]:
with EA.execution(configuration=configuration_record) as exec:
    print("An ML Execution.") 

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = configuration_record.execution_assets_path(asset_type_name)
# save assets to asset_path


# upload assets to catalog
EA.upload_execution(configuration_record, True)